In [1]:
%pip install beacon-api==1.0.9rc2


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from beacon_api import *
client = Client("https://beacon-cmems.maris.nl", jwt_token="")

Connected to: https://beacon-cmems.maris.nl/ server successfully


In [3]:
tables = client.list_tables()
tables

{'default': <beacon_api.table.DataTable at 0x7f249e4d5bb0>}

In [4]:
tables['default'].get_table_schema()

__entry_key: string
__chunk_index: uint64
TIME: timestamp[ns]
DEPTH: int64
LONGITUDE: float
LATITUDE: float
PRES: float
TRAJECTORY: string
TIME_QC: float
BATH: float
BATH_QC: float
POSITION_QC: float
PRES_QC: float
TEMP_DM: string
ACIDZZXX: float
PF12GCTX_QC: float
CPHL: float
DENS_QC: float
PHAEFLP1: float
ACIDZZXX_QC: float
NTRA_DM: string
DENS: float
CO2AQXXX-MLM3_QC: float
SLCA_DM: string
P113GCTX_QC: float
PSAL_DM: string
ALKYZZXX_QC: float
PHOS: float
NTRA_QC: float
SLCA: float
ALKYZZXX: float
PF12GCTX: float
TEMP_QC: float
TEMP: float
PSAL: float
PF11GCTX: float
NTRA: float
PHAEFLP1_QC: float
DOX1_QC: float
PSAL_QC: float
CPHL_QC: float
PHOS_QC: float
PF11GCTX_QC: float
PHOS_DM: string
NTRI_QC: float
SLCA_QC: float
CO2AQXXX-MLM3: float
NTRI: float
P113GCTX: float
DOX1: float
.project: string
.citation: string
.naming_authority: string
.license: string
.references: string
.update_interval: string
.Conventions: string
.comment: string
.history: string
.netcdf_version: string
.crea

In [5]:
query_builder = (
    tables['default']
    .query()
    .add_select_column("TIME", "COMMON_TIME")
    .add_select_column("TIME_QC", "COMMON_TIME_QC")
    .add_select_column("LONGITUDE")
    .add_select_column("LONGITUDE", "COMMON_LONGITUDE")
    .add_select(SelectLiteral("degree_east", "COMMON_LONGITUDE_UNITS"))
    .add_select(SelectLiteral("longitude", "COMMON_LONGITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALONZZ01", "COMMON_LONGITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGE", "COMMON_LONGITUDE_P06"))
    .add_select_column("LATITUDE")
    .add_select_column("LATITUDE", "COMMON_LATITUDE")
    .add_select(SelectLiteral("degree_north", "COMMON_LATITUDE_UNITS"))
    .add_select(SelectLiteral("latitude", "COMMON_LATITUDE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ALATZZ01", "COMMON_LATITUDE_P01"))
    .add_select(SelectLiteral("SDN:P06::DEGN", "COMMON_LATITUDE_P06"))
    .add_select_column("POSITION_QC")
    .add_select_column("POSITION_QC", "COMMON_POSITION_QC")
    
    .add_select_coalesced(["PRES_ADJUSTED", "PRES", "DEPH"], "COMMON_ORIGIN_DEPTH")
    .add_select_coalesced(["PRES_ADJUSTED_QC","PRES_QC","DEPH_QC"], "COMMON_ORIGIN_DEPTH_QC")
    .add_select_coalesced(["PRES_ADJUSTED.units", "PRES.units", "DEPH.units"], "COMMON_ORIGIN_DEPTH_UNITS")
    .add_select_coalesced(["PRES_ADJUSTED.standard_name", "PRES.standard_name", "DEPH.standard_name"], "COMMON_ORIGIN_DEPTH_STANDARD_NAME")
    .add_select(SelectFunction("coalesce_label", [SelectColumn("PRES"), SelectLiteral("SDN:P01::PRESPR01"), SelectColumn("DEPH"), SelectLiteral("SDN:P01::ADEPZZ01")], "COMMON_ORIGIN_DEPTH_P01"))
    .add_select(SelectFunction("coalesce_label", [SelectColumn("PRES"), SelectLiteral("SDN:P06::UPDB"), SelectColumn("DEPH"), SelectLiteral("SDN:P06::ULAA")], "COMMON_ORIGIN_DEPTH_P06"))
    
    .add_select(Functions.coalesce([Functions.map_pressure_to_depth(Functions.coalesce(["PRES_ADJUSTED", "PRES"],"pres"), "LATITUDE", "pres_harmonized"), "DEPH"], "COMMON_DEPTH"))
    .add_select_coalesced(["PRES_ADJUSTED_QC","PRES_QC","DEPH_QC"], "COMMON_DEPTH_QC")
    .add_select(SelectLiteral("m", "COMMON_DEPTH_UNITS"))
    .add_select(SelectLiteral("depth", "COMMON_DEPTH_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::ADEPZZ01", "COMMON_DEPTH_P01"))
    .add_select(SelectLiteral("SDN:P06::ULAA", "COMMON_DEPTH_P06"))
)

In [ ]:
query_builder = (
    query_builder
    .add_select_column("__entry_key")
    .add_select_column(".institution_edmo_code")
    .add_select_column(".institution_edmo_code", "COMMON_EDMO_CODE")
    .add_select(SelectLiteral("","COMMON_EDMERP_CODE"))
    .add_select_column(".featureType")
    .add_select_column(".featureType", "COMMON_FEATURE_TYPE")
    .add_select_column(".bigram")
    .add_select_column(".wmo_instrument_type")
    .add_select_column(".source_platform_category_code")
    .add_select(SelectFunction("concat", [SelectLiteral("SDN:L06::"), SelectColumn(".source_platform_category_code")], "COMMON_PLATFORM_L06"))
    .add_select_column(".id", "COMMON_ODV_TAG")
    # Set BEACON ref
    .add_select(SelectLiteral("BEACON_CMEMS_BGC", "SOURCE_BDI"))
    .add_select_column(".id","SOURCE_BDI_DATASET_ID")
)

In [7]:
# map chlorophyll
query_builder = (
    query_builder
    .add_select_column("CPHL")
    .add_select_column("CPHL_QC")
    .add_select_column("CPHL.units")
    .add_select_column("CPHL.standard_name")
    
    .add_select_column("CHLT")
    .add_select_column("CHLT_QC")
    .add_select_column("CHLT.units")
    .add_select_column("CHLT.standard_name")
    
    .add_select_coalesced(["CPHL", "CHLT"], "COMMON_ORIGIN_CHLOROPHYLL")
    .add_select_coalesced(["CPHL_QC", "CHLT_QC"], "COMMON_ORIGIN_CHLOROPHYLL_QC")
    .add_select_coalesced(["CPHL.units", "CHLT.units"], "COMMON_ORIGIN_CHLOROPHYLL_UNITS")
    .add_select_coalesced(["CPHL.standard_name", "CHLT.standard_name"], "COMMON_ORIGIN_CHLOROPHYLL_STANDARD_NAME")
    .add_select(SelectFunction("coalesce_label", [SelectColumn("CPHL"), SelectLiteral("SDN:P01::CPHLZZXX"), SelectColumn("CHLT"), SelectLiteral("SDN:P01::CHLTVOLU")], "COMMON_ORIGIN_CHLOROPHYLL_P01"))
    .add_select(SelectLiteral("SDN:P06::UMMC", "COMMON_ORIGIN_CHLOROPHYLL_P06"))
    
    .add_select_coalesced(["CPHL", "CHLT"], "COMMON_CHLOROPHYLL_PER_VOLUME")
    .add_select_coalesced(["CPHL_QC", "CHLT_QC"], "COMMON_CHLOROPHYLL_PER_VOLUME_QC")
    .add_select(SelectLiteral("mass_concentration_of_chlorophyll_in_sea_water", "COMMON_CHLOROPHYLL_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("mg m-3", "COMMON_CHLOROPHYLL_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::CHLTVOLU", "COMMON_CHLOROPHYLL_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UMMC", "COMMON_CHLOROPHYLL_PER_VOLUME_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_CHLOROPHYLL_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_CHLOROPHYLL_L22"))
)

In [8]:
# map oxygen
query_builder = (
    query_builder
    .add_select_column("DOXY")
    .add_select_column("DOXY_QC")
    .add_select_column("DOXY.units")
    .add_select_column("DOXY.standard_name")
    
    .add_select_column("DOX1")
    .add_select_column("DOX1_QC")
    .add_select_column("DOX1.units")
    .add_select_column("DOX1.standard_name")
    
    .add_select_column("DOX2")
    .add_select_column("DOX2_QC")
    .add_select_column("DOX2.units")
    .add_select_column("DOX2.standard_name")
    
    .add_select_coalesced(["DOXY", "DOX1", "DOX2"], "COMMON_ORIGIN_OXYGEN")
    .add_select_coalesced(["DOXY_QC", "DOX1_QC", "DOX2_QC"], "COMMON_ORIGIN_OXYGEN_QC")
    .add_select_coalesced(["DOXY.units", "DOX1.units", "DOX2.units"], "COMMON_ORIGIN_OXYGEN_UNITS")
    .add_select_coalesced(["DOXY.standard_name", "DOX1.standard_name", "DOX2.standard_name"], "COMMON_ORIGIN_OXYGEN_STANDARD_NAME")
    
    .add_select(SelectFunction("coalesce_label", [SelectColumn("DOXY"), SelectLiteral("SDN:P01::DOXYZZXX"), SelectColumn("DOX1"), SelectLiteral("SDN:P01::DOXYZZXX"), SelectColumn("DOX2"), SelectLiteral("SDN:P01::DOXMZZXX")], "COMMON_ORIGIN_OXYGEN_P01"))
    .add_select(SelectFunction("coalesce_label", [SelectColumn("DOXY"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("DOX1"), SelectLiteral("SDN:P06::UMLL"), SelectColumn("DOX2"), SelectLiteral("SDN:P06::KGUM")], "COMMON_ORIGIN_OXYGEN_P06"))

    .add_select(
        Functions.coalesce(
            [
                "DOXY",
                SelectFunction("map_units", [SelectLiteral("SDN:P06::UMLL"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("DOX1")]), 
                SelectFunction("map_units", [SelectLiteral("SDN:P06::KGUM"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("DOX2")])
            ],
        "COMMON_OXYGEN_PER_VOLUME"
        )
    )
    .add_select_coalesced(["DOXY_QC", "DOX1_QC", "DOX2_QC"], "COMMON_OXYGEN_PER_VOLUME_QC")
    .add_select(SelectLiteral("mole_concentration_of_dissolved_molecular_oxygen_in_sea_water", "COMMON_OXYGEN_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_OXYGEN_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXYZZXX", "COMMON_OXYGEN_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_OXYGEN_PER_VOLUME_P06"))
    
    .add_select(
        Functions.coalesce(
            [
                SelectFunction("map_units", [SelectLiteral("SDN:P06::UPOX"), SelectLiteral("SDN:P06::KGUM"), SelectColumn("DOXY")]), 
                SelectFunction("map_units", [SelectLiteral("SDN:P06::UMLL"), SelectLiteral("SDN:P06::KGUM"), SelectColumn("DOX1")]), 
                SelectColumn("DOX2")
            ],
        "COMMON_OXYGEN_PER_MASS"
        )
    )
    .add_select_coalesced(["DOXY_QC", "DOX1_QC", "DOX2_QC"], "COMMON_OXYGEN_PER_MASS_QC")
    .add_select(SelectLiteral("moles_of_oxygen_per_unit_mass_in_sea_water", "COMMON_OXYGEN_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_OXYGEN_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::DOXMZZXX", "COMMON_OXYGEN_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_OXYGEN_PER_MASS_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_OXYGEN_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_OXYGEN_L22"))
)

In [9]:
# map nitrate
query_builder = (
    query_builder
    .add_select_column("NTRA")
    .add_select_column("NTRA_QC")
    .add_select_column("NTRA.units")
    .add_select_column("NTRA.standard_name")
    
    .add_select_column("NTAW")
    .add_select_column("NTAW_QC")
    .add_select_column("NTAW.units")
    .add_select_column("NTAW.standard_name")
    
    .add_select_coalesced(["NTRA", "NTAW"], "COMMON_ORIGIN_NITRATE")
    .add_select_coalesced(["NTRA_QC", "NTAW_QC"], "COMMON_ORIGIN_NITRATE_QC")
    .add_select_coalesced(["NTRA.units", "NTAW.units"], "COMMON_ORIGIN_NITRATE_UNITS")
    .add_select_coalesced(["NTRA.standard_name", "NTAW.standard_name"], "COMMON_ORIGIN_NITRATE_STANDARD_NAME")
    .add_select(SelectFunction("coalesce_label", [SelectColumn("NTRA"), SelectLiteral("SDN:P01::NTRAZZXX"), SelectColumn("NTAW"), SelectLiteral("SDN:P01::MDMAP005")], "COMMON_ORIGIN_NITRATE_P01"))
    .add_select(SelectFunction("coalesce_label", [SelectColumn("NTRA"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("NTAW"), SelectLiteral("SDN:P06::KGUM")], "COMMON_ORIGIN_NITRATE_P06"))
    
    .add_select(
        Functions.coalesce(
            [
                "NTRA",
                SelectFunction("map_units", [SelectLiteral("SDN:P06::KGUM"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("NTAW")]),
            ]
            ,"COMMON_NITRATE_PER_VOLUME"
        )
    )
    .add_select_coalesced(["NTRA_QC", "NTAW_QC"], "COMMON_NITRATE_PER_VOLUME_QC")
    .add_select(SelectLiteral("mole_concentration_of_nitrate_in_sea_water", "COMMON_NITRATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_NITRATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::NTRAZZXX", "COMMON_NITRATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_NITRATE_PER_VOLUME_P06"))
    
    .add_select(
        Functions.coalesce(
            [
                SelectFunction("map_units", [SelectLiteral("SDN:P06::UPOX"), SelectLiteral("SDN:P06::KGUM"), SelectColumn("NTRA")]), 
                "NTAW",
            ],"COMMON_NITRATE_PER_MASS"
        )
    )
    .add_select_coalesced(["NTRA_QC", "NTAW_QC"], "COMMON_NITRATE_PER_MASS_QC")
    .add_select(SelectLiteral("moles_of_nitrate_per_unit_mass_in_sea_water", "COMMON_NITRATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_NITRATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP005", "COMMON_NITRATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_NITRATE_PER_MASS_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_NITRATE_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_NITRATE_L22"))
)

In [10]:
# MAP NITRATE + NITRITE
query_builder = (
    query_builder
    .add_select_column("NTRZ")
    .add_select_column("NTRZ_QC")
    .add_select_column("NTRZ.units")
    .add_select_column("NTRZ.standard_name")
    
    .add_select_column("NTRZ", "COMMON_ORIGIN_NITRATE_NITRITE")
    .add_select_column("NTRZ_QC", "COMMON_ORIGIN_NITRATE_NITRITE_QC")
    .add_select_column("NTRZ.units", "COMMON_ORIGIN_NITRATE_NITRITE_UNITS")
    .add_select_column("NTRZ.standard_name", "COMMON_ORIGIN_NITRATE_NITRITE_STANDARD_NAME")
    .add_select(SelectLiteral("SDN:P01::NTRZZZXX", "COMMON_ORIGIN_NITRATE_NITRITE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_NITRATE_NITRITE_P06"))
    
    .add_select_column("NTRZ", "COMMON_NITRATE_NITRITE_PER_VOLUME")
    .add_select_column("NTRZ_QC", "COMMON_NITRATE_NITRATE_PER_VOLUME_QC")
    .add_select(SelectLiteral("mole_concentration_of_nitrate_and_nitrite_in_sea_water", "COMMON_NITRATE_NITRITE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_NITRATE_NITRITE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::NTRZZZXX", "COMMON_NITRATE_NITRITE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_NITRATE_NITRITE_PER_VOLUME_P06"))
    
    .add_select(SelectFunction("map_units", [SelectLiteral("SDN:P06::UPOX"), SelectLiteral("SDN:P06::KGUM"), SelectColumn("NTRZ")], "COMMON_NITRATE_NITRITE_PER_MASS"))
    .add_select_column("NTRZ_QC", "COMMON_NITRATE_NITRITE_PER_MASS_QC")
    .add_select(SelectLiteral("moles_of_nitrate_and_nitrite_per_unit_mass_in_sea_water", "COMMON_NITRATE_NITRITE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_NITRATE_NITRITE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP006", "COMMON_NITRATE_NITRITE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_NITRATE_NITRITE_PER_MASS_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_NITRATE_NITRITE_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_NITRATE_NITRITE_L22"))
)

In [11]:
# map ammonium
query_builder = (
    query_builder
    .add_select_column("AMON")
    .add_select_column("AMON_QC")
    .add_select_column("AMON.units")
    .add_select_column("AMON.standard_name")
    
    .add_select_column("AMON", "COMMON_ORIGIN_AMMONIUM")
    .add_select_column("AMON_QC", "COMMON_ORIGIN_AMMONIUM_QC")
    .add_select_column("AMON.units", "COMMON_ORIGIN_AMMONIUM_UNITS")
    .add_select_column("AMON.standard_name", "COMMON_ORIGIN_AMMONIUM_STANDARD_NAME")
    .add_select(SelectLiteral("SDN:P01::AMONZZXX", "COMMON_ORIGIN_AMMONIUM_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_ORIGIN_AMMONIUM_P06"))
    
    .add_select_column("AMON", "COMMON_AMMONIUM_PER_VOLUME")
    .add_select_column("AMON_QC", "COMMON_AMMONIUM_PER_VOLUME_QC")
    .add_select(SelectLiteral("mole_concentration_of_ammonium_in_sea_water", "COMMON_AMMONIUM_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_AMMONIUM_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::AMONZZXX", "COMMON_AMMONIUM_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_AMMONIUM_PER_VOLUME_P06"))
    
    .add_select(SelectFunction("map_units", [SelectLiteral("SDN:P06::UPOX"), SelectLiteral("SDN:P06::KGUM"), SelectColumn("AMON")], "COMMON_AMMONIUM_PER_MASS"))
    .add_select_column("AMON_QC", "COMMON_AMMONIUM_PER_MASS_QC")
    .add_select(SelectLiteral("moles_of_ammonium_per_unit_mass_in_sea_water", "COMMON_AMMONIUM_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_AMMONIUM_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP005", "COMMON_AMMONIUM_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_AMMONIUM_PER_MASS_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_AMMONIUM_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_AMMONIUM_L22"))
)

In [12]:
# map phosphate
query_builder = (
    query_builder
    .add_select_column("PHOS")
    .add_select_column("PHOS_QC")
    .add_select_column("PHOS.units")
    .add_select_column("PHOS.standard_name")
    
    .add_select_column("PHOW")
    .add_select_column("PHOW_QC")
    .add_select_column("PHOW.units")
    .add_select_column("PHOW.standard_name")
    
    .add_select_coalesced(["PHOS", "PHOW"], "COMMON_ORIGIN_PHOSPHATE")
    .add_select_coalesced(["PHOS_QC", "PHOW_QC"], "COMMON_ORIGIN_PHOSPHATE_QC")
    .add_select_coalesced(["PHOS.units", "PHOW.units"], "COMMON_ORIGIN_PHOSPHATE_UNITS")
    .add_select_coalesced(["PHOS.standard_name", "PHOW.standard_name"], "COMMON_ORIGIN_PHOSPHATE_STANDARD_NAME")
    .add_select(SelectFunction("coalesce_label", [SelectColumn("PHOS"), SelectLiteral("SDN:P01::PHOSZZXX"), SelectColumn("PHOW"), SelectLiteral("SDN:P01::MDMAP906")], "COMMON_ORIGIN_PHOSPHATE_P01"))
    .add_select(SelectFunction("coalesce_label", [SelectColumn("PHOS"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("PHOW"), SelectLiteral("SDN:P06::KGUM")], "COMMON_ORIGIN_PHOSPHATE_P06"))
    
    .add_select(
        Functions.coalesce(
            [
                "PHOS",
                SelectFunction("map_units", [SelectLiteral("SDN:P06::KGUM"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("PHOW")]),
            ]
            ,"COMMON_PHOSPHATE_PER_VOLUME"
        )
    )
    .add_select_coalesced(["PHOS_QC", "PHOW_QC"], "COMMON_PHOSPHATE_PER_VOLUME_QC")
    .add_select(SelectLiteral("mole_concentration_of_phosphate_in_sea_water", "COMMON_PHOSPHATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_PHOSPHATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PHOSZZXX", "COMMON_PHOSPHATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_PHOSPHATE_PER_VOLUME_P06"))
    
    .add_select(
        Functions.coalesce(
            [
                SelectFunction("map_units", [SelectLiteral("SDN:P06::UPOX"), SelectLiteral("SDN:P06::KGUM"), SelectColumn("PHOS")]),
                "PHOW",
            ],"COMMON_PHOSPHATE_PER_MASS"
        )
    )
    .add_select_coalesced(["PHOS_QC", "PHOW_QC"], "COMMON_PHOSPHATE_PER_MASS_QC")
    .add_select(SelectLiteral("moles_of_phosphate_per_unit_mass_in_sea_water", "COMMON_PHOSPHATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_PHOSPHATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP906", "COMMON_PHOSPHATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_PHOSPHATE_PER_MASS_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_PHOSPHATE_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_PHOSPHATE_L22"))
)

In [13]:
# silicate
query_builder = (
    query_builder
    .add_select_column("SLCA")
    .add_select_column("SLCA_QC")
    .add_select_column("SLCA.units")
    .add_select_column("SLCA.standard_name")
    
    .add_select_column("SLCW")
    .add_select_column("SLCW_QC")
    .add_select_column("SLCW.units")
    .add_select_column("SLCW.standard_name")
    
    .add_select_coalesced(["SLCA", "SLCW"], "COMMON_ORIGIN_SILICATE")
    .add_select_coalesced(["SLCA_QC", "SLCW_QC"], "COMMON_ORIGIN_SILICATE_QC")
    .add_select_coalesced(["SLCA.units", "SLCW.units"], "COMMON_ORIGIN_SILICATE_UNITS")
    .add_select_coalesced(["SLCA.standard_name", "SLCW.standard_name"], "COMMON_ORIGIN_SILICATE_STANDARD_NAME")
    .add_select(SelectFunction("coalesce_label", [SelectColumn("SLCA"), SelectLiteral("SDN:P01::SLCAZZXX"), SelectColumn("SLCW"), SelectLiteral("SDN:P01::MDMAP012")], "COMMON_ORIGIN_SILICATE_P01"))
    .add_select(SelectFunction("coalesce_label", [SelectColumn("SLCA"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("SLCW"), SelectLiteral("SDN:P06::KGUM")], "COMMON_ORIGIN_SILICATE_P06"))
    
    .add_select(
        Functions.coalesce(
            [
                "SLCA", 
                SelectFunction("map_units", [SelectLiteral("SDN:P06::KGUM"), SelectLiteral("SDN:P06::UPOX"), SelectColumn("SLCW")]), 
            ]
            ,"COMMON_SILICATE_PER_VOLUME"
        )
    )
    .add_select_coalesced(["SLCA_QC", "SLCW_QC"],"COMMON_SILICATE_PER_VOLUME_QC")
    .add_select(SelectLiteral("mole_concentration_of_silicate_in_sea_water", "COMMON_SILICATE_PER_VOLUME_STANDARD_NAME"))
    .add_select(SelectLiteral("umol l-1", "COMMON_SILICATE_PER_VOLUME_UNITS"))
    .add_select(SelectLiteral("SDN:P01::SLCAZZXX", "COMMON_SILICATE_PER_VOLUME_P01"))
    .add_select(SelectLiteral("SDN:P06::UPOX", "COMMON_SILICATE_PER_VOLUME_P06"))
    
    .add_select(
        Functions.coalesce(
            [
                SelectFunction("map_units", [SelectLiteral("SDN:P06::UPOX"), SelectLiteral("SDN:P06::KGUM"), SelectColumn("SLCA")]),
                "SLCW",
            ],"COMMON_SILICATE_PER_MASS"
        )
    )
    .add_select_coalesced(["SLCA_QC", "SLCW_QC"],"COMMON_SILICATE_PER_MASS_QC")
    .add_select(SelectLiteral("moles_of_nitrate_per_unit_mass_in_sea_water", "COMMON_SILICATE_PER_MASS_STANDARD_NAME"))
    .add_select(SelectLiteral("umol kg-1", "COMMON_SILICATE_PER_MASS_UNITS"))
    .add_select(SelectLiteral("SDN:P01::MDMAP012", "COMMON_SILICATE_PER_MASS_P01"))
    .add_select(SelectLiteral("SDN:P06::KGUM", "COMMON_SILICATE_PER_MASS_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_SILICATE_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_SILICATE_L22"))
)

In [14]:
# map temperature
query_builder = (
    query_builder
    .add_select_column("TEMP")
    .add_select_column("TEMP_QC")
    .add_select_column("TEMP.units")
    .add_select_column("TEMP.standard_name")
    
    .add_select_column("TEMP_ADJUSTED")
    .add_select_column("TEMP_ADJUSTED_QC")
    .add_select_column("TEMP_ADJUSTED.units")
    .add_select_column("TEMP_ADJUSTED.standard_name")
    
    .add_select_coalesced(["TEMP", "TEMP_ADJUSTED"], "COMMON_ORIGIN_TEMPERATURE")
    .add_select_coalesced(["TEMP_QC", "TEMP_ADJUSTED_QC"], "COMMON_ORIGIN_TEMPERATURE_QC")
    .add_select_coalesced(["TEMP.units", "TEMP_ADJUSTED.units"], "COMMON_ORIGIN_TEMPERATURE_UNITS")
    .add_select_coalesced(["TEMP.standard_name", "TEMP_ADJUSTED.standard_name"], "COMMON_ORIGIN_TEMPERATURE_STANDARD_NAME")
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_ORIGIN_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_ORIGIN_TEMPERATURE_P06"))
    
    .add_select_coalesced(["TEMP", "TEMP_ADJUSTED"], "COMMON_TEMPERATURE")
    .add_select_coalesced(["TEMP_QC", "TEMP_ADJUSTED_QC"], "COMMON_TEMPERATURE_QC")
    .add_select(SelectLiteral("degree_C", "COMMON_TEMPERATURE_UNITS"))
    .add_select(SelectLiteral("sea_water_temperature", "COMMON_TEMPERATURE_STANDARD_NAME"))
    .add_select(SelectLiteral("SDN:P01::TEMPPR01", "COMMON_TEMPERATURE_P01"))
    .add_select(SelectLiteral("SDN:P06::UPAA", "COMMON_TEMPERATURE_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_TEMPERATURE_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_TEMPERATURE_L22"))
)

In [15]:
# map salinity
query_builder = (
    query_builder
    .add_select_column("PSAL")
    .add_select_column("PSAL_QC")
    .add_select_column("PSAL.units")
    .add_select_column("PSAL.standard_name")
    
    .add_select_column("PSAL_ADJUSTED")
    .add_select_column("PSAL_ADJUSTED_QC")
    .add_select_column("PSAL_ADJUSTED.units")
    .add_select_column("PSAL_ADJUSTED.standard_name")
    
    .add_select_coalesced(["PSAL", "PSAL_ADJUSTED"], "COMMON_ORIGIN_SALINITY")
    .add_select_coalesced(["PSAL_QC", "PSAL_ADJUSTED_QC"], "COMMON_ORIGIN_SALINITY_QC")
    .add_select_coalesced(["PSAL.units", "PSAL_ADJUSTED.units"], "COMMON_ORIGIN_SALINITY_UNITS")
    .add_select_coalesced(["PSAL.standard_name", "PSAL_ADJUSTED.standard_name"], "COMMON_ORIGIN_SALINITY_STANDARD_NAME")
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_ORIGIN_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_ORIGIN_SALINITY_P06"))
    
    .add_select_coalesced(["PSAL", "PSAL_ADJUSTED"], "COMMON_SALINITY")
    .add_select_coalesced(["PSAL_QC", "PSAL_ADJUSTED_QC"], "COMMON_SALINITY_QC")
    .add_select(SelectLiteral("sea_water_salinity", "COMMON_SALINITY_STANDARD_NAME"))
    .add_select(SelectLiteral("1e-3", "COMMON_SALINITY_UNITS"))
    .add_select(SelectLiteral("SDN:P01::PSLTZZ01", "COMMON_SALINITY_P01"))
    .add_select(SelectLiteral("SDN:P06::UUUU", "COMMON_SALINITY_P06"))
    
    # Set instrument type
    .add_select(SelectFunction("map_cora_instrument_l05", [SelectColumn(".wmo_instrument_type")], "COMMON_SALINITY_L05"))
    .add_select(SelectFunction("map_cora_instrument_l22", [SelectColumn(".wmo_instrument_type")], "COMMON_SALINITY_L22"))
)

In [16]:
import os 
import json

# Make output dir
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)
# for each year between 1950 and 2025
for year in range(1950, 2025):
    print(f"Processing year {year}")
    query_builder.filters = []
    query_builder = query_builder.add_range_filter("COMMON_TIME", f"{year}-01-01T00:00:00Z", f"{year}-12-31T23:59:59Z")
    query_builder = (
        query_builder
        .add_filter(OrFilter([
            IsNotNullFilter("COMMON_ORIGIN_CHLOROPHYLL"),
            IsNotNullFilter("COMMON_ORIGIN_OXYGEN"),
            IsNotNullFilter("COMMON_ORIGIN_NITRATE"),
            IsNotNullFilter("COMMON_ORIGIN_NITRATE_NITRITE"),
            IsNotNullFilter("COMMON_ORIGIN_AMMONIUM"),
            IsNotNullFilter("COMMON_ORIGIN_PHOSPHATE"),
            IsNotNullFilter("COMMON_ORIGIN_SILICATE"),
        ]))
    )
    output_file = os.path.join(output_dir, f"cmems_bgc_{year}.parquet")
    try:
        query_builder.to_parquet(output_file)
        print(f"Saved to {output_file}")
    except Exception as e:
        print(f"Failed to process year {year}: {e}")

Processing year 1950
Running query: {"from": "default", "select": [{"column": "TIME", "alias": "COMMON_TIME"}, {"column": "TIME_QC", "alias": "COMMON_TIME_QC"}, {"column": "LONGITUDE", "alias": null}, {"column": "LONGITUDE", "alias": "COMMON_LONGITUDE"}, {"value": "degree_east", "alias": "COMMON_LONGITUDE_UNITS"}, {"value": "longitude", "alias": "COMMON_LONGITUDE_STANDARD_NAME"}, {"value": "SDN:P01::ALONZZ01", "alias": "COMMON_LONGITUDE_P01"}, {"value": "SDN:P06::DEGE", "alias": "COMMON_LONGITUDE_P06"}, {"column": "LATITUDE", "alias": null}, {"column": "LATITUDE", "alias": "COMMON_LATITUDE"}, {"value": "degree_north", "alias": "COMMON_LATITUDE_UNITS"}, {"value": "latitude", "alias": "COMMON_LATITUDE_STANDARD_NAME"}, {"value": "SDN:P01::ALATZZ01", "alias": "COMMON_LATITUDE_P01"}, {"value": "SDN:P06::DEGN", "alias": "COMMON_LATITUDE_P06"}, {"column": "POSITION_QC", "alias": null}, {"column": "POSITION_QC", "alias": "COMMON_POSITION_QC"}, {"function": "coalesce", "args": [{"column": "PRES